# Measuring Personalization Performance Using Amplitude
In this exercise you will explore the conversion performance of your Personalize deployment in Retail Demo Store using [Amplitude's](https://amplitude.com/) real-time analytics platform.

*Recommended Time: 30 minutes*

## Prerequisites

This module uses [Amplitude](https://amplitude.com/) to measure the performance of the personalization features in the Retail Demo Store. It is assumed that you have either completed the Personalization workshop or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

This project also assumes that Amplitude has been configured during the deployment of the Retail Demo Store in your AWS account or managed event account.

In order to deploy Amplitude, you will need access to an Amplitude workspace, and an Amplitude API key that allows you to send data to Amplitude.  We recommend setting up a separate Amplitude project for the workshop, so that you can delete the demo data that will be passed to Amplitude as part of the workshop when you are finished.

## Overview

Deploying personalizaiton can be challenging in two ways:

1. Deciding where to deploy personalized experiences is often done using best guesses
2. Measuring the conversion outcomes of personalized experiences once they are deployed can be difficult without proper tooling

Amplitude addresses these issues by providing a comprehensive real-time measurement platform that allows you to track user behavior in the form of real-time events before and after the deployment of Personalize. 

In this workshop, you will set up tracking for Amplitude events, analyze user behavior prior to peronalization being deployed, and then measure the effects of personalization on user behavior after Personalize is deployed in the Retail Demo Store.

In the following steps, you will feed data about user behavior into Amplitude.  The dataset will include 60 days worth of simulated data going back from today.  The first 30 days will represent the time period before personalization is deployed, and the last thirty days will include user behavior data after personalization is deployed in the Retail Demo Store site.

## Amplitude Deployment Architecture

![Amplitude Deployment Architecture](./images/amplitude/workshop-architecture.png)

## Set Up Amplitude

Amplitude works by collecting real-time data from user behavior in the Retail Demo Store website.  The Amplitude tracking code is included in the workshop for you, so user interactions with the application will be collected in Amplitude if you included your Amplitude API key during the deployment of the Retail Demo Store environment.

Because the Retail Demo Store is developed using the Vue framework, we can load Amplitude when the user loads a page by including the Amplitude tracking script using an import statement in `../src/web-ui/src/analytics/AnalyticsHandler.js` (this file can be found in the Github repo for this project):

```
/* 
 * Centralized handling of all analytics calls for Pinpoint, Personalize 
 * (event tracker), and partner integrations.
 */
import Vue from 'vue';
import { Analytics as AmplifyAnalytics } from '@aws-amplify/analytics';
import Amplitude from 'amplitude-js'
```

Once the tracking script is loaded into a page, it can be used to fire events in response to user actions.  As an example, when a user clicks on a product on one of the Retail Demo Store pages, a ProductViewed event will be sent to Amplitude, using this code:

```
if (this.amplitudeEnabled()) {
    // Amplitude event
    var eventProperties = {
        productId: product.id,
        name: product.name,
        category: product.category,
        image: product.image,
        feature: feature,
        experimentCorrelationId: experimentCorrelationId,
        price: +product.price.toFixed(2)
    };
    Amplitude.getInstance().logEvent('ProductViewed', eventProperties);
}
```

These events are the same as events that are used to train the Personalize models used to offer users Personalize recommendations.  This is important since you will want the reports in Amplitude's analytics platform to reflect the same information that is being used to create Personalize models.  This will give you the most accurate way to measure user behavior before Personalize is deployed, and a consistent way to measure user behavior after Personalize is deployed in the Retail Demo Store site.

Retail Demo Store sends the following events to Personalize and Amplitude:

|Event Name|Event Description|Personalize|Amplitude|
|:-- |:-- |:-: |:-: |
|ProductAdded|Every time a user adds a product to their cart; tracks the ID of the product that was added|X|X|
|ProductRemoved|Every time a user removes a product from their cart||X|
|ProductQuantityUpdated|Every time a user adds an additional unit of a product|X|X|
|ProductViewed|Every time a user views a product detail page|X|X|
|CartViewed|Every time a user views the contents of their shopping cart|X|X|
|CheckoutStarted|Every time a user begins the checkout process|X|X|
|OrderCompleted|Every time a user completes a purchase successfully|X|X|

## Measuring Conversion Before Personalization is Deployed

Because Amplitude reports rely on real user data, you will be using a data simulator to create enough events for you to be able to run a useful report in Amplitude.  The data simulator included in the Retail Demo Store will send the events shown above as though they were being generated by real users of the Retail Demo Store.  This will simulate common browsing behaviors at a sufficent volume to allow you to run a report that looks like one that might be used in your own production applications.

The first simulated set of data will represent 30 days of user behavior in the Retail Demo Store prior to the deployment of the three personalization features you deployed in the first workshop exercise.  First, let's get some environment setup out of the way.

In [ ]:
import boto3
import os
import sys

sys.path.insert(0, os.path.abspath('../'))
!pip install -r ../requirements.txt


Then, let's get your Amplitude API key from SSM.  This will allow you to send simulated data directly to Amplitude.  If you did not set this up during your deployment of Retail Demo Store, skip to the next cell.

In [ ]:
ssm = boto3.client('ssm')

response = ssm.get_parameter(
    Name='retaildemostore-amplitude-api-key'
)

amplitude_config = {
    'api_key': response['Parameter']['Value']     # Do Not Change
}

print(amplitude_config)

If the cell above returned a valid Amplitude API key, you can run the cells after the next cell to populate your Amplitude project with data.

If you did not get a valid API key, it is likely you did not configure it in your deployment of Retail Demo Store.  You will need to set your Amplitude API key in the `amplitude_api_key` variable and then run the following cell to set the SSM key for Amplitude.

You will then need to re-deploy the web ui for the Retail Demo Store.

In [ ]:
# Set the Amplitude API key in the string below, and run this cell.  

# THIS IS ONLY REQUIRED IF YOU DID NOT SET THE AMPLITUDE API KEY IN YOUR ORIGINAL DEPLOYMENT

amplitude_api_key = ''

if amplitude_api_key:
    response = ssm.put_parameter(
        Name='retaildemostore-amplitude-api-key',
        Value='{}'.format(amplitude_api_key),
        Type='String',
        Overwrite=True
    )

    amplitude_config = {
        'api_key': '{}'.format(amplitude_api_key)
    }

print(amplitude_config)

## Demo Data Simulator

The code in the following cell implements a user simulator that mimics the behavior of real users.  The `usage_funnels_pre_personalize` array specifies a series of funnels that the simulator will use to generate events that are sent to Amplitude.  

The simulator uses the user profiles that are provided with the Retail Demo Store workshop, so that the user profiles shown in Amplitude reflect the same profiles you will see in the Retail Demo Store Users service.  Once a user profile is selected (at random), it will attempt to run the user through a selected funnel.  Not all users will complete each funnel, and some users may end up running through multiple funnels, depending on a weighted randomization function in the simulator.  This is done to provide realistic user behavior in the event patterns shown in Amplitude.  

The simulator will also create events based on a time of day algorithm, where a majority of users will end up being more active during business hours, and fewer users will be active during after hours.  User profiles are also mapped to particular product affinities in the Retail Demo Store catalog, so that it is possible to create user behaviors that are specific to certain kinds of product categories.



In [ ]:
import datetime
from datagenerator.rdscatalog import RDSCatalog
from datagenerator.rdsuserstate import RDSUserSelectionState
from datagenerator.users import UserPool
from datagenerator.sessions import Sessions
from datagenerator.output import OutputWriter

search_props_template = {
  'query': RDSUserSelectionState.user_search,
  'resultCount': RDSUserSelectionState.num_results,
  'reranked': False
}

search_props_feature_template = {
  'query': RDSUserSelectionState.user_search,
  'resultCount': RDSUserSelectionState.num_results,
  'reranked': True
}

prod_clicked_template = {
  'expand': RDSUserSelectionState.item_viewed_event_props
}

prod_clicked_feature_template = {
  'expand': RDSUserSelectionState.item_viewed_event_props,
  'feature': ['home_product_recs', 'product_detail_related']
}

prod_added_template = {
  'expand': RDSUserSelectionState.item_added_event_props,
}

checkout_start_template = {
  'expand': RDSUserSelectionState.cart_viewed_event_props,
}

order_completed_template = {
  'expand': RDSUserSelectionState.cart_viewed_event_props
}

catalog = RDSCatalog('../data/products.yaml')

usage_funnels_pre_personalize = [
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductSearched', search_props_template),
      ('ProductViewed', prod_clicked_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductsSearched', prod_clicked_template)
    ]
  }
]

date_time_now = datetime.datetime.now()

pool = UserPool.from_file('../data/users.json.gz')  # Load the known pool of users from the users file
print(f'Loaded {pool.size()} users...')
# Sends events to simulate application prior to personalization
sessions = Sessions(date_time_now - datetime.timedelta(days=60), date_time_now - datetime.timedelta(days=30), usage_funnels_pre_personalize, 100, pool)
writer = OutputWriter(sessions)
writer.to_amplitude(amplitude_config)
print(f'Events Send Completed')

## Create Amplitude Conversion Reports

Once the event generator has completed, log in to the [Amplitude console](https://analytics.amplitude.com).

In the console, you will use Amplitude's Funnel Analysis report to generate a report that enables you to see the conversion rate of the Retail Demo Store main purchase funnel.  Amplitude allows you to build a funnel for any user flow in your applications, but here we will focus on events leading to a purchase through the Retail Demo Store main purchase funnel. You will build a report that tracks all user sessions that perform the following actions in this order:

ProductViewed >
ProductAdded >
CheckoutStarted >
OrderCompleted

Note that users might perform other steps in the middle of this flow, such as viewing other products, adding and removing items from their cart, etc. but the Funnel report will ignore those events for the purpose of this report, since we are looking to see only conversions for users that have viewed a particular product and then purchased it through the minimum checkout flow steps for the Retail Demo Store.

![](images/amplitude/conversion-funnel-pre-setup.png)

To configure a Funnel report, select the New button in the left hand side of the screen, and then Report. Select Funnel Analysis for your report type at the top of the screen. In the events section below that, you will see a screen that looks like this:

![](images/amplitude/conversion-funnel-pre-settings.png)

Select the ProductViewed, ProductAdded, CheckoutStarted, and OrderCompleted events in that order.  Once that is complete, select the data dropdown on the lower right hand side of the screen, and select the Between tab. Then input a start date 60 days before today as a start and 30 days before today as an end date.  This report will show conversion data for the 30 days *prior* to personalization being deployed in the Retail Demo Store.  

![](images/amplitude/conversion-funnel-pre-dates.png)

Your funnel report will look something like this, though the exact conversion numbers and number of events will vary slightly from this screen shot:

![](images/amplitude/conversion-funnel-pre-pers.png)

Click on the lower part of the OrderCompleted bar graph. Note that the purchase conversion rate for the main funnel in the Retail Demo Store is 23.8%.  Not bad, but would we see an improvement in conversion if users were to receive personalized recommendations while they are interacting with the Retail Demo Store?


## Measuring the Conversion Effects of Personalized Product Recommendations

In the first workshop, you deployed the following Personalization features in the Retail Demo Store:

1. Personalized product recommendations
2. Similar product recommendations
3. Personalized ranking of products on category detail pages and personalized ranking of search results

In order to measure how users convert after interacting with these features, we need to add a parameter to the event tracking sent to Amplitude, so that our funnel report can differentiate between product views and purchases that are *not* influenced via personalization features and product views and purchases that are a result of user interaction with the newly-added personalization features.

In the `AnalyticsHandler.js` file, you will see code that adds a parameter to the ProductViewed event:

```
if (this.amplitudeEnabled()) {
    // Amplitude event
    var eventProperties = {
        productId: product.id,
        name: product.name,
        category: product.category,
        image: product.image,
        feature: feature,
        experimentCorrelationId: experimentCorrelationId,
        price: +product.price.toFixed(2)
    };
    Amplitude.getInstance().logEvent('ProductViewed', eventProperties);
}
```

The `feature` property will contain the name of the personalization feature that the user clicked on.  This will be used for personalized product recommendations and for similar product recommendations.  

This property will allow us to create a funnel report in Amplitude that shows the conversion rate of products clicked by users as a result of viewing a Personalization feature.

Run the following cell to generate data that will enable you to build a report that uses the `feature` property to analyze conversions after the personalization features have been deployed.


In [ ]:
usage_funnels_post_personalize_features = [
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductSearched', search_props_feature_template),
      ('ProductViewed', prod_clicked_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_feature_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  }
]

# Sends events to simulate application after personalization features are added
sessions = Sessions(date_time_now - datetime.timedelta(days=30), date_time_now, usage_funnels_post_personalize_features, 100, pool)
writer = OutputWriter(sessions)
writer.to_amplitude(amplitude_config)
print(f'Events Send Completed')

## Create Amplitude Personalization Conversion Reports 

Once the event generator has completed, go back to the Amplitude console.

In the console, you will once again use Amplitude's Funnel Analysis report to generate a report that enables you to see the conversion rate of the Retail Demo Store main purchase funnel.  This time after the Personalize features have been deployed.  We will still track the normal conversion funnel as before, but now we are going to drill down into ProductViewed events that were a result of the first two personalizaiton features we deployed earlier.

ProductViewed >
ProductAdded >
CheckoutStarted >
OrderCompleted

Instead of looking at all user paths through this funnel, we will look individually at the number of conversions that start with an interaction with a personalization feature.  Under the ProductViewed event, select `where` and then the `feature` property.  This will show a dropdown window that allows you to specify which values for the feature will be selected for the report.  In this case, we will look for events that are tied to 'product_detail_related' and 'home_product_recs'.  These values correspond to the Related Products feature, and the Product Recommendations feature for users that land on the home page.

![](./images/amplitude/product-viewed-feature-select.png)

This flow will now show conversions only for events that resulted in a purchase as a result of the user having initially looked at a product that was recommended on the home page (user personalization) or as a result of a related product recommendation (similar products personalization).

![](images/amplitude/conversion-funnel-post-pers.png)

This funnel shows a combined conversion rate of 30.6%.  This means that for users that have clicked on products that are recommended by these two features, there is a 6.8% increase in conversion (over the 23.8% conversion rate for for all users that view recommended or similar products over users that find products organically.

Speaking of organic searches, let's look at how personalized search results ranking performed against regular searches.

## Create Amplitude Search Personalization Conversion Reports

The Retail Demo Store allows users to perform a text search for products.  In the earlier part of the workshop, you deployed Personalize to re-sort product searches to show users products that might be more relevant based on their browsing history.

When the user performs a search in the Retail Demo Store, a ProductSearched event is sent:

```
if (this.amplitudeEnabled()) {
    // Amplitude event
    var eventProperties = {
        query: query,
        reranked: (user ? 'true' : 'false'),
        resultCount: numResults
    };
    Amplitude.getInstance().logEvent('ProductSearched', eventProperties);
}
```

If the search results are re-ranked by Personalize, the `reranked` property will be set to True in the event.  This means that we can compare user funnels from before and after Personalize is deployed.

For this funnel report, you will create an Amplitude funnel report that looks at:

ProductSearched >
ProductAdded >
CheckoutStarted >
OrderCompleted

First, let's look at search conversion before Personalize is deployed.  Set up a funnel report, with the events shown above.

![](images/amplitude/search-funnel-pre-setup.png)

Then select a time range that between 60 and 30 days before today (this is the time period prior to personalization being deployed in your dataset).

![](images/amplitude/search-funnel-pre-dates.png)

You should see a report that looks like this.

![](images/amplitude/search-funnel-pre-pers.png)

In this case, there is a 43% conversion rate for organic searches without personalized ranking.

Now, let's compare these results to search conversion after Personalize is deployed to re-rank search results.  In your funnel report, select the `where` option for the `ProductSearched` event, and then select the `reranked` property.  Select the `True` option.

![](images/amplitude/search-funnel-post-setup.png)

Then, select a time frame for the last 30 days, since this how long your dataset has personalization event data.

![](images/amplitude/search-funnel-post-dates.png)

Your report should now show conversions only for personalized events.

![](images/amplitude/search-funnel-post.png)



## Workshop Complete

Hopefully this introduction to user behavior monitoring using Amplitude showed you how you can deploy a similar solution along with Amazon Personalize. 

### Cleanup

Please remember to delete the Amplitude data that was sent to your Amplitude project when you are finished with it.
